---
format: 
  html:
    toc: false
    page-layout: full
execute:
    echo: false
---

## 2. Road Network/Walkability Analysis

In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
import cenpy
import pygris
import pandana as pnda
import osmnx as ox
import altair as alt
import pandana as pnda
import geoviews as gv
import geoviews.tile_sources as gvts
import warnings

from pandana.loaders import osm
from shapely.geometry import Point
from pandana.loaders import osm
from matplotlib import pyplot as plt

In [2]:
warnings.filterwarnings("ignore")

In [3]:
np.random.seed(42)
pd.options.display.max_columns = 999

A shorter walking distance to health centers ensures that community members have easier access to healthcare services. In emergency situations, such as accidents or sudden illnesses, quick access to a health center can be a matter of life and death. Shorter walk distances can improve emergency response times. When health centers are within walking distance, individuals are more likely to schedule and attend regular check-ups and screenings, contributing to preventive care and early detection of health issues. Therefore, we can conduct the walkability analysis for health centers in Philadelphia. Here, I used 2km as the walking distance.
I used the OSM network data and the corresponding API to do this osmnx analysis. The returning processed graph is shown with 199,029 nodes. I plotted the walking distance to the nearest health center for each point in the network. Please note that, due to quarto render memory restriction, I'm not able to upload some of the generated visulaizations to the website. Therefore, I'm inserting screenshots below to show the results that I ran in the Jupyter notebook. 

Please make sure to open the interactive html files called interactive_healcenter_distance.html and interactive_park_distance from the main folder to navigate the walkability analysis. I'm keeping the code for the visulizations below for demostration purpose. The complete code (larger file) is also available in the file called final project.ipynb.

In [4]:
philly = ox.geocode_to_gdf("Philadelphia, PA")

In [5]:
boundary = philly.bounds
boundary_float = [float(coord) for coord in boundary.values.flatten()]
boundary_float

[-75.2802977, 39.867005, -74.9558314, 40.1379593]

In [6]:
[lng_min, lat_min, lng_max, lat_max] = boundary_float

In [7]:
net = osm.pdna_network_from_bbox(
    lat_min, lng_min, lat_max, lng_max, network_type="walk"
)

Requesting network data within bounding box from Overpass API in 1 request(s)
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"motor|proposed|construction|abandoned|platform|raceway"]["foot"!~"no"]["pedestrians"!~"no"](39.86700500,-75.28029770,40.13795930,-74.95583140);>;);out;'}"
Downloaded 90,035.2KB from www.overpass-api.de in 3.56 seconds
Downloaded OSM network data within bounding box from Overpass API in 1 request(s) and 5.51 seconds
Returning OSM data with 577,585 nodes and 135,110 ways...
Edge node pairs completed. Took 111.99 seconds
Returning processed graph with 199,029 nodes and 291,436 edges...
Completed OSM data download and Pandana node and edge table creation in 125.12 seconds


In [8]:
# sensible defaults
max_distance = 2000  # in meters
num_pois = 10  # only need the 10 nearest POI to each point in the network

In [9]:
Health_Centers_gdf = gpd.read_file("./data/Health_Centers.geojson")

In [10]:
Health_Centers_gdf = Health_Centers_gdf.to_crs(epsg=4326)
Health_Centers_gdf['lon'] = Health_Centers_gdf.geometry.x
Health_Centers_gdf['lat'] = Health_Centers_gdf.geometry.y

In [11]:
net.set_pois('health_centers', max_distance, num_pois, Health_Centers_gdf['lon'], Health_Centers_gdf['lat'])

# Access the nearest health center for each point in the network
access = net.nearest_pois(distance=max_distance, category='health_centers', num_pois=num_pois)


In [12]:
# keyword arguments to pass for the matplotlib figure
bbox_aspect_ratio = (lat_max - lat_min) / (lng_max - lng_min)
fig_kwargs = {"facecolor": "w", "figsize": (10, 10 * bbox_aspect_ratio)}

# keyword arguments to pass for scatter plots
plot_kwargs = {"s": 20, "alpha": 0.9, "cmap": "viridis_r", "edgecolor": "none"}

In [13]:
access.tail(n=50)

,1,2,3,4,5,6,7,8,9,10
id,,,,,,,,,,
11434816809,217.279007,1427.699951,1693.581055,1712.387939,2000.0,2000.0,2000.0,2000.0,2000.0,2000.0
11434816810,220.731995,1424.246948,1690.128052,1715.840942,2000.0,2000.0,2000.0,2000.0,2000.0,2000.0
11434816811,437.075012,1246.297974,1748.411011,1837.038940,2000.0,2000.0,2000.0,2000.0,2000.0,2000.0
11435381643,2000.000000,2000.000000,2000.000000,2000.000000,2000.0,2000.0,2000.0,2000.0,2000.0,2000.0
11435381645,2000.000000,2000.000000,2000.000000,2000.000000,2000.0,2000.0,2000.0,2000.0,2000.0,2000.0
11435381646,2000.000000,2000.000000,2000.000000,2000.000000,2000.0,2000.0,2000.0,2000.0,2000.0,2000.0
11435381648,2000.000000,2000.000000,2000.000000,2000.000000,2000.0,2000.0,2000.0,2000.0,2000.0,2000.0
11435381650,2000.000000,2000.000000,2000.000000,2000.000000,2000.0,2000.0,2000.0,2000.0,2000.0,2000.0
11435381651,2000.000000,2000.000000,2000.000000,2000.000000,2000.0,2000.0,2000.0,2000.0,2000.0,2000.0


In [14]:
# Merge the nodes and the distance to POIs
nodes = pd.merge(net.nodes_df, access, left_index=True, right_index=True)

# Make into a geodataframe
nodes = gpd.GeoDataFrame(
    nodes, geometry=gpd.points_from_xy(nodes["x"], nodes["y"]), crs="EPSG:4326"
)

In [15]:
def plot_walking_distance(net, health_centers, distance=1000, n=1):
    
    from mpl_toolkits.axes_grid1 import make_axes_locatable


    # get the distances to nearest num_pois POI
    access = net.nearest_pois(distance=1000, category='health_centers', num_pois=num_pois)

    # merge node positions and distances to nearest PO
    nodes = pd.merge(net.nodes_df, access, left_index=True, right_index=True)
    nodes = gpd.GeoDataFrame(
        nodes, geometry=gpd.points_from_xy(nodes["x"], nodes["y"]), crs="EPSG:4326"
    )

    # Create the figure
    fig, ax = plt.subplots(figsize=(10, 10))
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1)

    # plot the distance to the nth nearest amenity
    ax = nodes.plot(ax=ax, cax=cax, column=nodes[n], legend=True, **plot_kwargs)

    # add the amenities as stars
    for i, row in Health_Centers_gdf.iterrows():
        ax.scatter(row["lon"], row["lat"], color="red", s=100, marker="*")

    # format
    ax.set_facecolor("black")
    ax.figure.set_size_inches(fig_kwargs["figsize"])

    # set extent
    [xmin, ymin, xmax, ymax] = nodes.geometry.total_bounds
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)

    return ax

In [16]:
#ax = plot_walking_distance(net, "health_centers", n=1)
#ax.set_title("Walking distance to the nearest health center", fontsize=18);

![HealthCenter](./data/Static_healthcenter.png)

In the map above, each star represents a health center in Philadelphia, and the blue-yellow colors around them represents distances in the surrounding area that has a walking distance (2km) to the closest health center. Since it is a static map, the result may not be the most ideal for visulization, so I include an interactive visulization as shown below.

In [17]:
import folium
from folium.plugins import MarkerCluster

def plot_walking_distance_interactive(net, health_centers, distance=1000, n=1, plot_kwargs={}, fig_kwargs={}):
    """
    Plot the walking distance to the specified amenity in an interactive Folium map.
    """
    # Get the distances to the nearest num_pois POI
    access = net.nearest_pois(distance=distance, category=health_centers, num_pois=n)

    # Merge node positions and distances to the nearest POI
    nodes = pd.merge(net.nodes_df, access, left_index=True, right_index=True)
    nodes['geometry'] = gpd.points_from_xy(nodes['x'], nodes['y'])
    nodes = gpd.GeoDataFrame(nodes, geometry='geometry', crs="EPSG:4326")

    # Create the Folium map
    m = folium.Map(location=[nodes.geometry.y.mean(), nodes.geometry.x.mean()], zoom_start=14, **fig_kwargs)

    # Plot the distance to the nth nearest amenity
    mc = MarkerCluster()
    for _, row in nodes.iterrows():
        folium.CircleMarker(
            location=[row['geometry'].y, row['geometry'].x],
            radius=5,
            color='blue',
            fill=True,
            fill_color='blue',
            fill_opacity=0.5,
            tooltip=f"Distance: {row[n]} meters"
        ).add_to(mc)

    # Add the amenities as stars
    for _, row in Health_Centers_gdf.iterrows():
        folium.Marker([row["lat"], row["lon"]], popup=row["NAME"], icon=folium.Icon(color='red', icon='star')).add_to(m)

    # Add the MarkerCluster to the map
    m.add_child(mc)

    return m


In [18]:
#m = plot_walking_distance_interactive(net, "health_centers", n=1)
#m

In [19]:
#m.save("interactive_healthcenter_distance.html")

![health_center_interface](./data/healthcenter_detail.png)

I used folium and OpenStreetMap to build this interactive visulization. From this map, each red pin represents a health center, and you can see the health center name by clicking it. You can also zoom in and click the nodes see their distance to the nearest health center.

Similarly, walking distance to park can also be worthwhile to explore. Parks provide spaces for physical activities and exercise. Short walk distances to parks encourage community members to engage in regular physical activity, promoting overall fitness and reducing the risk of lifestyle-related diseases. 

Using the same method as for the health centers, I plotted the walking distance to the nearest park for each node in the network. 

In [20]:
url = "https://opendata.arcgis.com/datasets/d52445160ab14380a673e5849203eb64_0.geojson"
parks = gpd.read_file(url).to_crs("EPSG:2272")

In [21]:
# Get the centroids of each polygon
centroids = parks['geometry'].centroid

# Create a new GeoDataFrame with point geometries
park_centroids = gpd.GeoDataFrame(geometry=centroids, crs=parks.crs)
park_centroids['PUBLIC_NAME'] = parks['PUBLIC_NAME']

park_centroids = park_centroids.to_crs(epsg=4326)
park_centroids['lon'] = park_centroids.geometry.x
park_centroids['lat'] = park_centroids.geometry.y

# Sensible defaults
max_distance = 2000  # in meters
num_pois = 10  # find the single nearest POI to each point in the network

# Set the POIs using the lon and lat columns of Health_Centers_gdf
net.set_pois('park', max_distance, num_pois, park_centroids['lon'], park_centroids['lat'])

# Access the nearest health center for each point in the network
access1 = net.nearest_pois(distance=max_distance, category='park', num_pois=num_pois)

In [22]:
access1.tail(n=50)

,1,2,3,4,5,6,7,8,9,10
id,,,,,,,,,,
11434816809,986.976990,1176.223999,1216.720947,1279.937988,1334.058960,1341.866943,1480.754028,1506.970947,1786.937988,1824.391968
11434816810,990.429993,1172.770996,1220.173950,1283.390991,1330.605957,1338.413940,1484.207031,1510.423950,1790.390991,1820.938965
11434816811,897.911011,959.763977,1140.327026,1180.077026,1203.543945,1219.886963,1404.359985,1417.905029,1792.785034,1879.222046
11435381643,203.257996,879.161011,998.744995,1280.339966,1358.546021,1750.884033,1784.796997,1814.337036,2000.000000,2000.000000
11435381645,210.488007,870.616028,1027.373047,1308.968018,1387.173950,1742.338989,1776.251953,1842.964966,2000.000000,2000.000000
11435381646,130.220001,1102.963989,1223.863037,1504.644043,1583.663940,1951.130005,1974.687012,2000.000000,2000.000000,2000.000000
11435381648,124.037003,1092.645020,1213.543945,1494.324951,1573.344971,1961.448975,1964.368042,1998.281006,2000.000000,2000.000000
11435381650,76.792000,1118.063965,1238.963013,1470.906006,1598.764038,1966.644043,1969.756958,1989.786987,2000.000000,2000.000000
11435381651,91.055000,1091.558960,1189.813965,1375.491943,1549.614990,1855.473999,1858.587036,1963.281982,1997.194946,2000.000000


In [23]:
# Merge the nodes and the distance to POIs
nodes1 = pd.merge(net.nodes_df, access1, left_index=True, right_index=True)

# Make into a geodataframe
nodes1 = gpd.GeoDataFrame(
    nodes1, geometry=gpd.points_from_xy(nodes1["x"], nodes1["y"]), crs="EPSG:4326"
)

In [24]:
def plot_walking_distance(net, park, distance=1000, n=1):
    """
    Plot the walking distance to the specified amenity
    """
    from mpl_toolkits.axes_grid1 import make_axes_locatable


    # get the distances to nearest num_pois POI
    access1 = net.nearest_pois(distance=1000, category='park', num_pois=num_pois)

    # merge node positions and distances to nearest PO
    nodes1 = pd.merge(net.nodes_df, access1, left_index=True, right_index=True)
    nodes1 = gpd.GeoDataFrame(
        nodes1, geometry=gpd.points_from_xy(nodes1["x"], nodes1["y"]), crs="EPSG:4326"
    )

    # Create the figure
    fig, ax = plt.subplots(figsize=(10, 10))
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1)

    # plot the distance to the nth nearest amenity
    ax = nodes1.plot(ax=ax, cax=cax, column=nodes1[n], legend=True, **plot_kwargs)

    # add the amenities as stars
    for i, row in park_centroids.iterrows():
        ax.scatter(row["lon"], row["lat"], color="red", s=100, marker="*")

    # format
    ax.set_facecolor("black")
    ax.figure.set_size_inches(fig_kwargs["figsize"])

    # set extent
    [xmin, ymin, xmax, ymax] = nodes1.geometry.total_bounds
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)

    return ax

In [25]:
#ax = plot_walking_distance(net, "park", n=1)
#ax.set_title("Walking distance to the nearest park", fontsize=18);

![Park](./data/static_Park.png)

In the map above, each star represents a park in Philadelphia, and the blue-yellow colors around them represents distances in the surrounding area that has a walking distance (2km) to the closest park. Since it is a static map, the result may not be the most ideal for visulization, so I include an interactive visulization as shown below.

In [26]:
def plot_walking_distance_interactive(net, park, distance=1000, n=1, plot_kwargs={}, fig_kwargs={}):
    """
    Plot the walking distance to the specified amenity in an interactive Folium map.
    """
    # Create the Folium map
    m1 = folium.Map(location=[nodes1.geometry.y.mean(), nodes1.geometry.x.mean()], zoom_start=14, **fig_kwargs)

    # Plot the distance to the nth nearest amenity
    mc1 = MarkerCluster()
    for _, row in nodes1.iterrows():
        folium.CircleMarker(
            location=[row['geometry'].y, row['geometry'].x],
            radius=5,
            color='blue',
            fill=True,
            fill_color='blue',
            fill_opacity=0.5,
            tooltip=f"Distance: {row[n]} meters"
        ).add_to(mc1)

    # Add the amenities as stars
    for _, row in park_centroids.iterrows():
        folium.Marker([row["lat"], row["lon"]], popup=row["PUBLIC_NAME"], icon=folium.Icon(color='red', icon='star')).add_to(m1)

    # Add the MarkerCluster to the map
    m1.add_child(mc1)

    return m1


In [27]:
#m1 = plot_walking_distance_interactive(net, "park", n=1)
#m1

![park_interface](./data/park_detail.png)

I used folium and OpenStreetMap to build this interactive visulization. From this map, each red pin represents a health center, and you can see the health center name by clicking it. You can also zoom in and click the nodes see their distance to the nearest health center.

In [28]:
#m1.save("interactive_park_distance.html")